In [1]:
!pip install transformers torch

In [2]:
# 2. Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 3. Импорт необходимых модулей
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# 4. Путь к модели на Google Drive
model_path = "/content/drive/MyDrive/gemmaX2/"

# 5. Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # Используем 16-bit точность для экономии памяти
    device_map="auto"           # Автоматически распределяем модель по доступным устройствам
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# 6. Переводим модель в режим оценки
model.eval()

def generate_response(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            repetition_penalty=1.2,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [81]:
system_prompt = """
Ты - система информационного поиска НИУ ВШЭ.
Создай краткий ответ на основе предоставленных документов.
Используй только информацию из источников.
Отвечай на русском языке, не повторяйся.
"""

question = "Как получить бесплатную психологическую помощь в НИУ ВШЭ"
contexst = """Психологическая помощь и телефоны горячих линий\n
Горячая линия для студентов и аспирантов НИУ ВШЭ работает круглосуточно.
По телефону 8-800-250-04-33 или через чат можно получить консультацию психолога."""

user_query = f"""
{question}
=========
{contexst}
"""

full_prompt = f"{system_prompt}\nВопрос: {user_query}\nОтвет:"


response = generate_response(full_prompt)
print(response.split("Ответ:")[-1].strip())

Для получения бесплатной психологической помощи в Научно-техническом институте Высшей школы экономики (НИУ ВШЭ) вы можете обратиться к специалистам учебного центра "Разумный Возраст", который находится внутри самого Института. Учебный центр "Разумный возраст" предлагает различные виды занятий: от индивидуальных консультаций до групповых семинаров по различным темам. Кроме того, у вас есть возможность воспользоваться сервисом виртуальной реальности под названием Virtual Reality Experience System , где вам предложат ряд игр и развлечений с использованием технологий дополненной реальности.


In [83]:
def generate_hse_answer(question: str, context: str):
  system_prompt = """
  Ты - система информационного поиска НИУ ВШЭ.
  Создай краткий ответ на основе предоставленных документов.
  Используй только информацию из источников.
  Отвечай на русском языке, не повторяйся.
  """
  user_query = f"""
  {question}
  =========
  {context}
  """

  full_prompt = f"{system_prompt}\nВопрос: {user_query}\nОтвет:"


  response = generate_response(full_prompt)
  return(response.split("Ответ:")[-1].strip())

In [84]:
context = """Психологическая помощь и телефоны горячих линий\n
Горячая линия для студентов и аспирантов НИУ ВШЭ работает круглосуточно.
По телефону 8-800-250-04-33 или через чат можно получить консультацию психолога."""
question = "Как получить психологическую помощь?"

answer = generate_hse_answer(
    question=question,
    context=context
)

print(f"Вопрос: {question}\nОтвет: {answer}")

Вопрос: Как получить психологическую помощь?
Ответ: Пожалуйста, обратите внимание на пункты 1) и 9). Для получения помощи по вопросам обучения, научной деятельности и других вопросов учебного характера обращайтесь в Студенческий совет НИУ ВШЭ (СтуН), который предоставляет услуги социально ориентированного консультирования студентам и аспирантам как в режиме онлайн, так и лично. Чтобы обратиться за помощью к Стунду, нажмите ссылку "Обратиться за поддержкой" во вкладке "Вопросы и ответы", которая находится слева от главного меню сайта.
